In [ ]:
import os
from openai import OpenAI

In [ ]:
test_data_dir = './test_dataset'
test_result_dir = './test_results/gpt4_0' # test_result_dir = './test_results/gpt3_5' 

prompt_type_list = ["zero_shot_template", "few_shot_template", "CoT_CSP_template", "zero_shot_CSP_template"]

api_key = 'your api key'
gt_type_list = ["defect", "clean"]

number_list = [str(num) for num in range(1, 31)]

start_str = " : start===================================================\n"
end_str = "\nend===================================================\n\n\n\n\n\n\n\n\n\n\n\n\n"

In [ ]:
def get_content(template_file):
    with open(template_file, "r", encoding="utf-8") as f:
        return f.read()
        
if __name__ == "__main__":
    for prompt_type in prompt_type_list:
        for gt_type in gt_type_list:
            result_file_name = test_result_dir + '/' + gt_type + '_' + prompt_type + '_' + 'result.txt'
            with open(result_file_name, "w", encoding="utf-8") as f:
                for data_num in number_list:
                    input_dir = test_data_dir + '/' + data_num + '/' + gt_type + '/' + data_num + '_' + gt_type + '_' + prompt_type + ".txt"
                    print(input_dir)
                    input_prompt = get_content(input_dir)
                    ret = ""
                    # LLM api part start
                    client = OpenAI(api_key= api_key)
                    question = input_prompt
                    completion = client.chat.completions.create(
                        model="gpt-4", # gpt-3.5-turbo-0125, gpt-3.5-turbo-instruct, gpt-4, gpt-4-32k
                        messages=[
                            {'role': 'system', 'content': "You are a helpful assistant."},
                            {'role': 'user', 'content': question}
                        ]
                    )
                    response = completion.choices[0].message.content
    
                    if prompt_type == "CoT_CSP_template":
                        ret = data_num + start_str + response
                        ret = ret + end_str
                    else:
                        ret = ret + response
                        ret = ret.replace('Label: ', '')
                        ret = ret + '\n'
                    f.write(ret)
                # LLM api part end
        
    print("done!")